# Тестовое задание

Требуется разработать алгоритм максимизации функции k на ориентированном мультиграфе (ССиТ).

Построение алгоритма будет выполняться в 2 этапа:
1. Написание движка для нахождения функции k на заданном ориентированном мультиграфе (ССиТ)
2. Оптимизация функции k

### Этап 1: Написание движка для нахождения функции k на заданном ориентированном мультиграфе

In [ ]:
from sympy import symbols, Eq, solve

#Создание класса для вершин и ребер графа
# class Well:
#     def __init__(self, flow_rate, pressure, alpha, const):
#         self.flow_rate = 
#         self.pressure = 
#         self.alpha = alpha
#         self.const = const
class Well:
    def __init__(self, idx, alpha, const, Q_well):
        self.idx = idx
        # self.p = p
        self.Q_well = Q_well
        self.alpha = alpha
        self.const = const
    def flow_rate(self):
        return self.alpha * (  ) * self.const
          
class Vertex:
    def __init__(self, idx):
        self.idx = idx
        # self.edge_input = edge_input # Список труб, что входит в узел
        self.Q = 0
        # self.Q = ((self.Q + edge_input[i].Q_edge) for i in range(len(edge_input))) # Список весов в узле
        # self.output_flow = output_flow
    def Q_Vertex(self, edge_input):
        self.Q = ((self.Q + edge_input[i].Q_edge) for i in range(len(edge_input)))
        return self.Q

class Edge:
    def __init__(self, idx, vertex1, vertex2, length, diameter, roughness, density):
        self.idx = idx
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.length = length
        self.diameter = diameter
        self.roughness = roughness
        self.Q_edge = self.vertex1
        self.density = density
        self.pressure_loss = self.density * self.length * self.Q_edge * self.roughness / self.diameter
    # def pressure_loss(self, density):
    #     return density * self.length * self.Q_edge * self.roughness / self.diameter

class Drain:
    def __init__(self, idx, p_0, alpha, const):
        self.idx = idx
        self.p_0 = p_0
        self.alpha = alpha
        self.const = const
        self.Q_0 = self.alpha * self.p_0 + self.const

In [ ]:
# Тело Алгоритма
alpha = 3
const = 5
density = 1000

def create_graph():
    # get the number of Wells
    num_wells = int(input("Enter the number of Wells: "))
    wells = []
    # Создание списка из num_wells элементов - Q1, Q2, Q3
    flow_rate = []
    for i in range(num_wells):
        flow_rate.append('Q' + str(i))
    flow_rate = symbols(flow_rate)
    
    for i in range(num_wells):
        wells.append(Well(i+1, alpha, const, flow_rate[i]))

    # get the number of Vertex
    num_vertex = int(input("Enter the number of Vertex: "))
    vertex = []
    for i in range(num_wells):
        vertex.append(Vertex(i+1, ))

    num_vertex = symbols(flow_rate)
    # get the number of Edges
    num_edges = int(input("Enter the number of Edges: "))
    edges = []
    for i in range(num_edges):
    #     vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1))) - 1
    #     vertex2 = int(input("Enter the second vertex for edge {}: ".format(i + 1))) - 1
        length = float(input("Enter the length for edge {}: ".format(i + 1)))
        diameter = float(input("Enter the diameter for edge {}: ".format(i + 1)))
        roughness = float(input("Enter the roughness for edge {}: ".format(i + 1)))
        edges.append(Edge(wells[vertex1], wells[vertex2], length, diameter, roughness))

    # return wells, edges

In [25]:
class Node:
    def __init__(self, idx):
        self.idx = idx
        self.adjacent_nodes = []
        self.adjacent_edges = []
    
    def add_edge(self, node, edge):
        self.adjacent_nodes.append(node)
        self.adjacent_edges.append(edge)
        
class Edge:
    def __init__(self, idx, node1, node2, x, D, mu):
        self.idx = idx
        self.start_node = node1
        self.end_node = node2
        self.x = x
        self.D = D
        self.mu = mu
    
class Well(Node):
    def __init__(self, idx, Q, p):
        Node.__init__(self, idx)
        self.Q = Q
        self.p = p
        
class Graph:
    def __init__(self, num_nodes, num_edges, num_wells):
        self.nodes = []
        self.edges = []
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.num_wells = num_wells
        self.wells = []
        
    def add_node(self, node):
        self.nodes.append(node)
        
    def add_edge(self, edge):
        self.edges.append(edge)
        
    def add_well(self, well):
        self.wells.append(well)
        
    def create_graph(self):
        graph = {}
        for node in self.nodes:
            graph[node.idx] = []
            for i in range(len(node.adjacent_nodes)):
                graph[node.idx].append((node.adjacent_nodes[i].idx, node.adjacent_edges[i].idx))
        return graph
        
# # Reading input
# num_wells = int(input("Enter the number of wells: "))
# num_nodes = num_wells + int(input("Enter the number of additional nodes: "))
# num_edges = int(input("Enter the number of edges: "))

# graph = Graph(num_nodes, num_edges, num_wells)

# Adding wells
# print("Enter the number of wells:")
# num_wells = int(input().strip())
# wells = []
# for i in range(num_wells):
#     print("Enter flow rate and pressure for well", i+1, "separated by a space:")
#     flow_rate, pressure = map(int, input().strip().split())
#     wells.append((flow_rate, pressure))

# # Adding nodes
# print("Enter the number of nodes:")
# num_nodes = int(input().strip())

# # Adding edges
# print("Enter the number of edges:")
# num_edges = int(input().strip())
# graph = {}
# for i in range(num_edges):
#     print("Enter vertex 1 and vertex 2 for edge", i+1, "separated by a space:")
#     vertex1, vertex2 = map(int, input().strip().split())
#     if vertex1 not in graph:
#         graph[vertex1] = []
#     graph[vertex1].append(vertex2)

# print("Graph:", graph)

Enter the number of wells:
Enter flow rate and pressure for well 1 separated by a space:
Enter flow rate and pressure for well 2 separated by a space:
Enter flow rate and pressure for well 3 separated by a space:
Enter flow rate and pressure for well 4 separated by a space:
Enter the number of nodes:
Enter the number of edges:
Enter vertex 1 and vertex 2 for edge 1 separated by a space:
Enter vertex 1 and vertex 2 for edge 2 separated by a space:
Enter vertex 1 and vertex 2 for edge 3 separated by a space:


ValueError: not enough values to unpack (expected 2, got 0)

## Тестовая часть, понадлежит удалению перед отправкой

In [7]:
from sympy import symbols, Eq, solve

x, y = symbols('x y')

class MyExpression:
    def __init__(self):
        self.expr = x + y
        
    def evaluate(self, x_value, y_value):
        return self.expr.subs([(x, x_value), (y, y_value)])

def evaluate_expression(expr, x_value, y_value):
    return expr.subs([(x, x_value), (y, y_value)])

expr = MyExpression()
# print(evaluate_expression(expr, 2, 3)) # Output: 5
print(expr.evaluate(2, 3)) # Output: 5

5


In [1]:
from sympy import symbols, Eq, solve


variables = ['x', 'y', 'z']
variables = symbols(variables)
print(variables)
print(type(variables[0]))

variables = ['x', 'y', 'z']
x, y, z = symbols(','.join(variables))
print(x, y, z)
print(type(x))


[x, y, z]
<class 'sympy.core.symbol.Symbol'>
x y z
<class 'sympy.core.symbol.Symbol'>


In [ ]:
input().strip().split()
print(a)

In [5]:
b = []
for i in range(3):
        b.append('Q' + str(i))

print(b)

['Q0', 'Q1', 'Q2']
